In [18]:
import pandas as pd
import numpy as np
import os
import math

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly_football_pitch import (
    make_pitch_figure,
    PitchDimensions,
    SingleColourBackground
)

from tqdm import tqdm
import ast

import dash
from dash import no_update
from dash import dcc, html
from dash.dependencies import Input, Output

import warnings
warnings.filterwarnings("ignore")

In [19]:
def calculate_position(x, y, distance, angle_rad):
    """
    Calculate the (x, y) position of loc1 given the distance and angle from loc2.
    
    Parameters:
    loc2 (tuple): The reference point (x2, y2).
    distance (float): The distance between loc1 and loc2.
    angle (float): The angle between the line connecting loc1 and loc2 and the positive x-axis.
    
    Returns:
    tuple: The (x, y) coordinates of loc1.
    """
    # Calculate the x and y coordinates
    new_x = x + distance * np.cos(angle_rad)
    new_y = y + distance * np.sin(angle_rad)

    return [new_x, new_y]

In [20]:
action_df = pd.read_csv('../data/processed/clean_action_data_glob_v2.csv')
# for i in range(len(action_df)):
#     if i%2 != 0:
#         action_df.drop(i, axis = 0, inplace = True)
        
genetic_df = pd.read_csv('../src/modelling/genetic/results/result_each_gen.csv')
genetic_df['initial_xg'] = genetic_df['initial_xg'].astype(float)

In [21]:
action_df.head()

,minute,period,shot_open_goal,shot_statsbomb_xg,fk_duration,pass_angle,pass_length,pass_switch,shot_x,shot_y,...,teammates_player_5,teammates_player_6,teammates_player_7,teammates_player_8,teammates_player_9,teammates_player_10,action_id,pred_base_xg,pred_improved_xg,pctge_improvement
0,37,1,1,0.047790,1.418802,1.062347,35.947323,1,109.3,36.4,...,1.0,0.0,0.0,1,1,0,0,0.047803,0.116530,143.770004
1,48,1,1,0.058214,1.031450,-2.230199,21.382704,1,101.5,44.6,...,0.0,0.0,1.0,0,1,0,1,0.059697,0.100624,68.559998
2,74,2,1,0.014924,2.715643,0.891994,53.833538,1,104.3,55.1,...,0.0,0.0,0.0,1,0,0,2,0.015474,0.058507,278.109985
3,51,2,1,0.057205,1.154653,0.984665,29.648777,1,106.2,36.9,...,1.0,0.0,0.0,0,0,0,3,0.052976,0.168502,218.070007
4,49,2,1,0.085958,1.555444,1.689729,32.026237,1,112.1,41.4,...,0.0,1.0,0.0,1,0,1,4,0.085725,0.159260,85.779999


In [22]:
genetic_df.head()

,index,idx_generation,best_solution,initial_xg,best_xG
0,0,1,[109.3 36.4 114.83774618601626 38.068798494898...,0.047803,[0.06066725]
1,0,2,[112.1 37.70889994669599 113.71281406309255 35...,0.047803,[0.10649709]
2,0,3,[112.1 37.70889994669599 113.71281406309255 35...,0.047803,[0.10649709]
3,0,4,[112.1 37.70889994669599 113.71281406309255 35...,0.047803,[0.10649709]
4,0,5,[112.1 38.27305314769811 113.71281406309255 35...,0.047803,[0.10855474]


In [23]:
complete_df = genetic_df.merge(action_df, left_on = 'index', right_on = 'action_id', how = 'inner')

In [24]:
complete_df = complete_df[['index', 'action_id', 'idx_generation', 'best_solution', 'initial_xg', 'best_xG', 'shot_statsbomb_xg', 
    'shot_x', 'shot_y', 'fk_x', 'fk_y', 'distance_player_1', 'distance_player_2', 'distance_player_3', 'distance_player_4', 'distance_player_5', 'distance_player_6',
    'distance_player_7', 'distance_player_8', 'distance_player_9', 'distance_player_10', 
    'angle_player_1', 'angle_player_2', 'angle_player_3', 'angle_player_4', 'angle_player_5', 'angle_player_6', 'angle_player_7', 'angle_player_8', 'angle_player_9', 'angle_player_10',
    'teammates_player_1', 'teammates_player_2', 'teammates_player_3', 'teammates_player_4', 'teammates_player_5', 'teammates_player_6',
    'teammates_player_7', 'teammates_player_8', 'teammates_player_9', 'teammates_player_10', 'pred_base_xg', 'pred_improved_xg', 'pctge_improvement']]

In [25]:
# retrieve all players position
for i in tqdm(range(1, 11)):
    complete_df[f'position_player_{i}'] = [calculate_position(complete_df['shot_x'][j], 
                                                              complete_df['shot_y'][j], 
                                                              complete_df[f'distance_player_{i}'][j], 
                                                              complete_df[f'angle_player_{i}'][j]) 
                                            for j in range(len(complete_df))]
    complete_df.drop([f'distance_player_{i}', f'angle_player_{i}'], axis = 1, inplace = True)

100%|██████████| 10/10 [00:01<00:00,  9.85it/s]


In [26]:
# keep only opponents fixed pos
for i in tqdm(range(1, 11)):
    if complete_df[f'teammates_player_{i}'][0] == 1:
        complete_df.drop([f'position_player_{i}', f'teammates_player_{i}'], axis = 1, inplace = True)

100%|██████████| 10/10 [00:00<00:00, 1207.48it/s]


In [27]:
# format best solution
complete_df['best_solution'] = [complete_df['best_solution'][j].replace('\n', '').replace(' ', ', ') for j in range(len(complete_df))]
complete_df['best_solution'] = [ast.literal_eval(complete_df['best_solution'][j]) for j in range(len(complete_df))]

complete_df['shot_pos_gen_x'] = [complete_df['best_solution'][j][0] for j in range(len(complete_df))]
complete_df['shot_pos_gen_y'] = [complete_df['best_solution'][j][1] for j in range(len(complete_df))]

complete_df['pos_teammate_1_x'] = [complete_df['best_solution'][j][2] if len(complete_df['best_solution'][j]) >= 3 else None for j in range(len(complete_df))]
complete_df['pos_teammate_2_x'] = [complete_df['best_solution'][j][4] if len(complete_df['best_solution'][j]) >= 5 else None for j in range(len(complete_df))]
complete_df['pos_teammate_3_x'] = [complete_df['best_solution'][j][6] if len(complete_df['best_solution'][j]) >= 7 else None for j in range(len(complete_df))]
complete_df['pos_teammate_4_x'] = [complete_df['best_solution'][j][8] if len(complete_df['best_solution'][j]) >= 9 else None for j in range(len(complete_df))]
complete_df['pos_teammate_5_x'] = [complete_df['best_solution'][j][10] if len(complete_df['best_solution'][j]) >= 11 else None for j in range(len(complete_df))]

complete_df['pos_teammate_1_y'] = [complete_df['best_solution'][j][3] if len(complete_df['best_solution'][j]) >= 4 else None for j in range(len(complete_df))]
complete_df['pos_teammate_2_y'] = [complete_df['best_solution'][j][5] if len(complete_df['best_solution'][j]) >= 6 else None for j in range(len(complete_df))]
complete_df['pos_teammate_3_y'] = [complete_df['best_solution'][j][7] if len(complete_df['best_solution'][j]) >= 8 else None for j in range(len(complete_df))]
complete_df['pos_teammate_4_y'] = [complete_df['best_solution'][j][9] if len(complete_df['best_solution'][j]) >= 10 else None for j in range(len(complete_df))]
complete_df['pos_teammate_5_y'] = [complete_df['best_solution'][j][11] if len(complete_df['best_solution'][j]) >= 12 else None for j in range(len(complete_df))]

i=1
for col in complete_df.columns:
    if col.startswith('position'):
        complete_df[f'pos_opponent_{i}_x'] = [complete_df[col][j][0] for j in range(len(complete_df))]
        complete_df[f'pos_opponent_{i}_y'] = [complete_df[col][j][1] for j in range(len(complete_df))]
        complete_df.drop(col, axis = 1)
        i += 1

complete_df.drop(['best_solution'], axis = 1, inplace = True)

# format best_xg
complete_df['best_xG'] = complete_df['best_xG'].apply(lambda x : float(ast.literal_eval(x)[0]))

In [28]:
complete_df.isna().sum()['pos_teammate_5_y'] # number of rows without 5 teammates

6700

In [38]:
# Function to create the football pitch using plotly_football_pitch
def create_pitch():
    
    dimensions = PitchDimensions(pitch_width_metres=80, pitch_length_metres=120)

    fig = make_pitch_figure(
        dimensions,
        pitch_background=SingleColourBackground("#3ab54a"),
    )

    fig.update_layout(
        xaxis=dict(range=[0, 120], showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(range=[0, 80], showgrid=False, zeroline=False, showticklabels=False),
        plot_bgcolor='#3ab54a',
        width=600, height=400,
        margin=dict(l=0, r=0, b=0, t=0)
    )

    return fig

# Function to add scatter points to the plot
def add_scatter(fig, data_row, show_legend=False):
    nb_opponents = 0
    nb_teammates = 0
    for col in list(data_row.index):
        if col.startswith('pos_opponent'):
            nb_opponents+=0.5
        elif col.startswith('pos_teammate'):
            nb_teammates+=0.5
    
    # Shot and free-kick positions
    fig.add_trace(go.Scatter(x=[data_row['shot_pos_gen_x']], y=[data_row['shot_pos_gen_y']], mode='markers',
                             marker=dict(size=10, symbol='star', color='red'), name='Shot Position', showlegend=show_legend))
    fig.add_trace(go.Scatter(x=[data_row['fk_x']], y=[data_row['fk_y']], mode='markers',
                             marker=dict(size=10, symbol='diamond', color='purple'), name='Freekick Position', showlegend=show_legend))

    # Opponents' positions
    for i in range(1, int(nb_opponents)):
        fig.add_trace(go.Scatter(x=[data_row[f'pos_opponent_{i}_x']], y=[data_row[f'pos_opponent_{i}_y']], mode='markers',
                                 marker=dict(size=8, symbol='x', color='orange'), name=f'Opponent', showlegend=show_legend and i == 1))

    # Teammates' positions
    for i in range(1, int(nb_teammates)):
        fig.add_trace(go.Scatter(x=[data_row[f'pos_teammate_{i}_x']], y=[data_row[f'pos_teammate_{i}_y']], mode='markers',
                                 marker=dict(size=8, symbol='circle', color='blue'), name=f'Teammate', showlegend=show_legend and i == 1))

    return fig

# Function to update the pitch with progress line and scatter points
def update_pitch(frame, df):
    data_row = df.iloc[frame]
    fig = create_pitch()

    # Add scatter points
    fig = add_scatter(fig, data_row, show_legend=(frame == 0))

    return fig

# Function to update the line plot with new points
def update_line_plot(frame, df, line_fig):
    if frame == 0:
        # Add the first point with no connecting line
        line_fig.add_trace(go.Scatter(
            x=[0],  # Frame 0 as x-axis
            y=[df['pred_base_xg'].iloc[0]],  # First y value
            mode='lines+markers',
            name='Line Plot',
            marker=dict(color='#f0543c')
        ))
    else:
        # Ensure that the trace exists before updating it
        if len(line_fig.data) > 0:
            # Append the next point to both x and y lists
            line_fig.data[0].x = list(line_fig.data[0].x) + [frame]
            line_fig.data[0].y = list(line_fig.data[0].y) + [df['best_xG'].iloc[frame]]

            if frame == 1:
                line_fig.data[0].update(mode='lines+markers')

    return line_fig



In [39]:
data_path = os.path.join(os.pardir,'data', 'processed', 'clean_action_data_glob_v2.csv')

df = pd.read_csv(data_path)
df = df[df['action_id'].isin(complete_df.action_id.unique())]

overall_improv_list = list(filter(lambda x: not math.isnan(x), df['pctge_improvement'].to_list()))
overall_fitness_list = list(filter(lambda x: not math.isnan(x), df['pred_improved_xg'].to_list()))
overall_basexg = list(filter(lambda x: not math.isnan(x), df['pred_base_xg'].to_list()))

In [40]:
app = dash.Dash(__name__)

# Create figure using Plotly graph objects
fig = make_subplots()

# Add scatter plot
scatter = go.Scatter(
    x=overall_basexg, 
    y=overall_fitness_list, 
    mode='markers', 
    marker=dict(color='blue', size=8),
    name='Improved xG'
)
fig.add_trace(scatter)

# Add line plot for overall_basexg vs overall_basexg (identity line)
line = go.Scatter(
    x=overall_basexg, 
    y=overall_basexg, 
    mode='lines', 
    line=dict(color='black'),
    name='Base xG'
)
fig.add_trace(line)

# Update layout
fig.update_layout(
    title='Base xG VS Improvement',
    xaxis_title='Base xG',
    yaxis_title='Improved xG',
    showlegend=True,
    template='plotly_white',
    width=800,
    height=800
)


# Layout of the app
app.layout = html.Div([
    dcc.Graph(id='scatter-plot', 
              figure=fig),
    dcc.Graph(id='dedicated-plot')  
])

@app.callback(
    Output('dedicated-plot', 'figure'),
    Input('scatter-plot', 'clickData')
)
def display_dedicated_plot(clickData):
    if clickData is None:
        fig = go.Figure()

        fig.update_layout(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            annotations=[
                {
                    'text': "Click on a point to display the animation<br>of the genetic algorithm results",
                    'xref': 'paper',
                    'yref': 'paper',
                    'showarrow': False,
                    'font': {'size': 18, 'color': 'black'},
                    'align': 'center',
                    'x': 0.5,
                    'y': 0.01,
                }
            ],
            width=700,
            height=500,
            template='plotly_white'
        )
        return fig
    
    # Extract data from clicked point
    clicked_point = clickData['points'][0]
    index = clicked_point['pointIndex']
    sub_df = complete_df[complete_df['action_id'] == index]

    # Creating frames for animation
    frames = []
    pitch_fig = create_pitch()
    line_fig = go.Figure()

    for frame in range(len(sub_df)):
        pitch_fig = update_pitch(frame, sub_df)
        line_fig = update_line_plot(frame, sub_df, line_fig)
        frames.append(go.Frame(data=pitch_fig.data + line_fig.data, name=str(frame)))

    # Creating the initial pitch and line plot
    initial_pitch_fig = create_pitch()
    initial_pitch_fig = add_scatter(initial_pitch_fig, sub_df.iloc[0], show_legend=True)

    # Adding frames to the figure for animation
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=('Football Pitch', 'Evolution of xG across Generations'),
        specs=[[{"type": "scatter"}], [{"type": "scatter"}]],
        vertical_spacing=0.1
    )

    # Add initial pitch and line plot
    fig.add_traces(initial_pitch_fig.data, rows=1, cols=1)
    fig.add_traces(line_fig.data, rows=2, cols=1)

    fig.frames = frames

    # Animation settings
    fig.update_layout(
        updatemenus=[{
            "buttons": [
                {
                    "args": [None, {"frame": {"duration": 500, "redraw": False}, "fromcurrent": True}],
                    "label": "Play",
                    "method": "animate"
                },
                {
                    "args": [[None], {"frame": {"duration": 500, "redraw": False}, "mode": "immediate", "transition": {"duration": 0}}],
                    "label": "Pause",
                    "method": "animate"
                }
            ],
            "direction": "left",
            "pad": {"r": 10, "t": 87},
            "showactive": False,
            "type": "buttons",
            "x": 0.1,
            "xanchor": "right",
            "y": 0,
            "yanchor": "top"
        }],
        sliders=[{
            "yanchor": "top",
            "xanchor": "left",
            "currentvalue": {
                "font": {"size": 20},
                "prefix": "Frame:",
                "visible": True,
                "xanchor": "right"
            },
            "transition": {"duration": 500, "easing": "cubic-in-out"},
            "pad": {"b": 10, "t": 50},
            "len": 0.9,
            "x": 0.1,
            "y": 0,
            "steps": [{
                "args": [[str(k)], {"frame": {"duration": 500, "redraw": False}, "mode": "immediate",
                                    "transition": {"duration": 500}}],
                "label": str(k),
                "method": "animate",
            } for k in range(len(frames))]
        }],
        xaxis1=dict(range=[0, 120], showgrid=False, zeroline=False, showticklabels=False),
        yaxis1=dict(range=[0, 80], showgrid=False, zeroline=False, showticklabels=False),
        shapes=[
            dict(
                type="rect",
                xref="x1", yref="y1",
                x0=0, y0=0, x1=120, y1=80,
                fillcolor="#3ab54a",
                layer="below",
                line_width=0,
            ),
            dict(
                type="circle",
                xref="x1", yref="y1",
                x0=60 - 10, y0=40 - 10,
                x1=60 + 10, y1=40 + 10,
                line_color="black",
                line_width=3
            )
        ],
        xaxis2=dict(tickvals=list(range(0, len(sub_df), 5)), ticktext=list(range(0, len(sub_df), 5))),
        xaxis2_title='Generation #',
        yaxis2_title='xG',
        width=700,  # Adjusted for narrower width
        height=900  # Adjusted for increased height
    )

    return fig
app.layout = html.Div([
    html.Div(
        dcc.Graph(id='scatter-plot', figure=fig),
        style={'width': '50%', 'display': 'inline-block'}
    ),
    html.Div(
        dcc.Graph(id='dedicated-plot'),  # Placeholder for the dedicated plot
        style={'width': '50%', 'display': 'inline-block'}
    ),
], style={'display': 'flex'})

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port = '8070')